In [1]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

In [2]:
def load_data():
    df=open('./train.txt','r')
    text=df.read()
    corpus=text.split('\n')
    xtrain=[]
    ytrain=[]
    for x in corpus:
        temp=x.split(';')
        if(len(temp)==2):
            xtrain.append(temp[0])
            ytrain.append(temp[1])
    df=pd.DataFrame([xtrain,ytrain])
    df.to_csv('./train.csv')
    

load_data()

In [3]:
def processing_test():
    df=open('./test.txt','r')
    text=df.read()
    corpus=text.split('\n')
    xtrain=[]
    ytrain=[]
    for x in corpus:
        temp=x.split(';')
        if(len(temp)==2):
            xtrain.append(temp[0])
            ytrain.append(temp[1])
    print(len(xtrain))
    print(len(ytrain))
    df=pd.DataFrame([xtrain,ytrain])
    df.to_csv('./test.csv')

processing_test()


2000
2000


In [4]:
def load():
    df=pd.read_csv("./train.csv")
    xtrain=df.iloc[0]
    ytrain=df.iloc[1]
    ytrain=ytrain.replace(['sadness','anger','love','surprise','fear','joy'],[1,2,3,4,5,6])
    return np.array(xtrain),np.array(ytrain)
xtrain,ytrain=load()
print(xtrain)

[0 'i didnt feel humiliated'
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'
 ... 'i feel strong and good overall'
 'i feel like this was such a rude comment and im glad that t'
 'i know a lot but i feel so stupid because i can not portray it']


In [5]:
xtrain=xtrain[1:]
ytrain=ytrain[1:]

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
stpwrd=stopwords.words('english')
lemma=WordNetLemmatizer()
for i in range(len(xtrain)):
    sent=xtrain[i]
    words=word_tokenize(sent)
    words=[lemma.lemmatize(w,'v') for w in words if(w not in stpwrd)]
    xtrain[i]=' '.join(words)
print(xtrain)

['didnt feel humiliate'
 'go feel hopeless damn hopeful around someone care awake'
 'im grab minute post feel greedy wrong' ... 'feel strong good overall'
 'feel like rude comment im glad' 'know lot feel stupid portray']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
xtrain=vectorizer.fit_transform(xtrain)

In [67]:
xtrain=xtrain.toarray()
print(xtrain)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [9]:
print(xtrain.shape)
ytrain[ytrain==6]=0

(16000, 12118)


In [10]:
len(np.unique(ytrain))
print(np.bincount(ytrain))

[5362 4666 2159 1304  572 1937]


In [11]:
print(np.bincount(ytrain))

[5362 4666 2159 1304  572 1937]


In [12]:
print(xtrain.shape)
print(ytrain.shape)

(16000, 12118)
(16000,)


In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [50]:
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
print(xtrain)

[[-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]
 [-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]
 [-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]
 ...
 [-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]
 [-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]
 [-0.0135433  -0.00790594 -0.00790594 ... -0.00790594 -0.01363595
  -0.00790594]]


In [51]:
x1,x2,y1,y2=train_test_split(xtrain,ytrain,train_size=.8,random_state=20)

In [52]:
y1.shape

(12800,)

In [53]:
print(x1.shape)
print(np.bincount(y1))
print(np.unique(y1))

(12800, 12118)
[4291 3717 1738 1033  450 1571]
[0 1 2 3 4 5]


In [57]:
import numpy as np
import pandas as pd

def var_creation(x):
    row,col=x.shape
    w1=np.random.randn(100,row)*np.sqrt(2/row)
    b1=np.random.randn(100,1)*np.sqrt(2/row)
    w2=np.random.randn(6,100)*np.sqrt(2/100)
    b2=np.random.randn(6,1)*np.sqrt(2/100)
    return w1,b1,w2,b2
def relu(z1):
    return np.maximum(z1,0)
def softmax(z2):
    return np.exp(z2)/sum(np.exp(z2))
def forward_propogation(x,w1,b1,w2,b2):
    z1=w1.dot(x)+b1
    a1=relu(z1)
    z2=w2.dot(a1)+b2
    a2=softmax(z2)
    return z1,a1,z2,a2
def onehot(y):
    output=np.zeros((6,len(y)))
    for i in range(len(y)):
        output[y[i]][i]=1
    return output
def deri_relu(z1):
    return z1>0
def backward_propogation(w2,y,a1,z1,a2,z2,x):
    onehot_y=onehot(y)
    sample=x.shape[1]
    dl2=a2-onehot_y
    dl1=(((w2.T).dot(dl2))*deri_relu(z1))
    dj_w2=(1/sample)*(dl2.dot(a1.T))
    dj_b2=(1/sample)*np.sum(dl2,axis=1).reshape(6,1)
    dj_w1=(1/sample)*dl1.dot(x.T)
    dj_b1=(1/sample)*np.sum(dl1,axis=1).reshape(100,1)
    return dj_w2,dj_b2,dj_w1,dj_b1

def update_params(w1,w2,b1,b2,alpha,dj_w2,dj_b2,dj_w1,dj_b1):
    w1=w1-alpha*(dj_w1)
    b1=b1-alpha*(dj_b1)
    w2=w2-alpha*(dj_w2)
    b2=b2-alpha*(dj_b2)
    return w1,b1,w2,b2

def calc_acc(pred,y):
    pred=pred.T
    pred_res=[np.argmax(x) for x in pred]
    print(np.bincount(pred_res))
    print(np.sum(pred_res==y)/len(y))
    
def gradient(epoc,alpha,x,y):
    b=.9
    w1,b1,w2,b2=var_creation(x)
    v1,v2=0,0
    for _ in range(epoc):
        z1,a1,z2,a2=forward_propogation(x,w1,b1,w2,b2)
        dj_w2,dj_b2,dj_w1,dj_b1=backward_propogation(w2,y,a1,z1,a2,z2,x)
        v1=b*v1+(1-b)*dj_w1
        v2=b*v2+(1-b)*dj_w2
        w1,b1,w2,b2=update_params(w1,w2,b1,b2,alpha,v2,dj_b2,v1,dj_b1)
        if(_%10==0):
            calc_acc(a2,y)
    return w1,b1,w2,b2



In [64]:
xtrain_temp=x1.T
w1,b1,w2,b2=gradient(180,.02,xtrain_temp,y1)

[2161 2989  157 4374 2703  416]
0.166484375
[3807 4138  270 2524 1505  556]
0.246640625
[5608 4733  435  926  452  646]
0.35125
[5811 4868  701  459  148  813]
0.42890625
[5426 4900 1020  370   90  994]
0.502734375
[5219 4762 1238  402   65 1114]
0.5759375
[5224 4618 1272  455   72 1159]
0.64140625
[5227 4546 1275  507   75 1170]
0.6978125
[5165 4490 1285  577   94 1189]
0.74390625
[5101 4425 1317  627  112 1218]
0.7815625
[5018 4368 1368  653  127 1266]
0.81140625
[4953 4304 1434  677  145 1287]
0.834453125
[4869 4245 1482  709  170 1325]
0.857890625
[4791 4192 1521  738  185 1373]
0.8771875
[4730 4138 1544  772  208 1408]
0.89265625
[4680 4091 1570  798  226 1435]
0.905234375
[4627 4051 1595  817  253 1457]
0.91609375
[4594 4020 1616  830  274 1466]
0.92390625


In [65]:
print(w1)

[[-0.01993365 -0.01073839 -0.00868795 ...  0.00033719  0.00690864
   0.02814274]
 [ 0.00405608  0.00350373  0.01332168 ... -0.01589763 -0.00344752
  -0.00256196]
 [-0.0012343  -0.00553418 -0.01545036 ... -0.00844522  0.01384931
  -0.02414283]
 ...
 [-0.00795813  0.00889641  0.00113047 ...  0.01614534  0.0198948
  -0.01619456]
 [ 0.00092256  0.0069544   0.01280045 ...  0.01380648  0.02362708
  -0.0013271 ]
 [-0.01410275 -0.00949606 -0.00565828 ... -0.00605556 -0.01027954
  -0.02803557]]


In [68]:
z1,a1,z2,a2=forward_propogation(x2.T,w1,b1,w2,b2)
print(np.bincount(y2))
calc_acc(a2,y2)

[1071  949  421  271  122  366]
[1336 1090  331  146   36  261]
0.5834375
